# Intro

You have the tools to obtain data from a single table in whatever format you want it. But what if the data you want is spread across multiple tables?

That's where **JOIN** comes in! **JOIN** is incredibly important in practical SQL workflows. So let's get started.

# Example

We'll use our imaginary pets dataset, but we'll add a second table. The first table, "pets", has three columns, with information on 
- The ID number of each pet
- The pet's name, and 
- The type of animal 

The "owners" table also has three columns: 
- An ID number for each owner (different from the Pet ID number)
- The name of the owner
- The ID number of the pet (which matches the ID number in the "pets" table.

![](https://i.imgur.com/W4gYoNg.png)

Each row in each table is associated with a single pet, and we refer to the same pets in both tables. That's why there are two columns (`ID` in the "pets" table and `Pet_ID` in the "owners" table) that have the same information in them: the ID number of the pet. 

We can match rows with the same value in these columns to get information that applies to a certain pet. For example, the "pets" table shows that the pet with the ID 1 is named Dr. Harris Bonkers. The "owners" table shows that "Aubrey Little" is the owner of the pet with Pet ID 1. Putting these two facts together, Dr. Harris Bonkers is owned by Aubrey Little. 

Fortunately, we don't have to do this by hand to figure out which owner's name goes with which pet name. **JOIN** allows us to create a new table combining information from the "pets" and "owners" tables. For example, we can create a table with just two columns: the name of the pet and the name of the owner. 

![](https://i.imgur.com/zqQdJTI.png)

The syntax to create that table looks like this:

    SELECT p.Name AS Pet_Name, o.Name as Owner_Name
    FROM `bigquery-public-data.pet_records.pets` as p
    INNER JOIN `bigquery-public-data.pet_records.owners` as o 
        ON p.ID = o.Pet_ID

Notice that since the `ID` column exists in both datasets, we have to clarify which one to use. When you're joining tables, it's a good habit to specify which table each of your columns comes from. That way you don't have to pull up the schema every time you go back to read the query.

The type of **JOIN** we're using today is called an **INNER JOIN**. That means that a row will only be put in the final output table if the value in the column you're using to combine them shows up in both the tables you're joining. For example, if Tom's ID code of 4 didn't exist in the "pets" table, we would only get 3 rows back from this query. There are other types of **JOIN**, but an **INNER JOIN** is very widely used, so it's a good one to start with.

> **What does "ON" do?** It says which column in each table to look at to combine the tables. Here were using the `ID` column from the "pets" table and the `Pet_ID` column from the "owners" table.

Now that we've talked about the concept behind using **JOIN**, let's work through an example together.

# Example: How many files are covered by each type of software license?

GitHub is the most popular place to collaborate on software. A GitHub "repo" is a collection of files associated with a specific project.  

Most repos on GitHub are shared under a specific legal license, which determines the legal restrictions on how they are used.  For our example, we're going to look at how many different files have been released under each license. 

First, of course, we need to get our environment ready to go:

In [ ]:
# import package with helper functions 
import bq_helper

# create a helper object for this dataset
github = bq_helper.BigQueryHelper(active_project="bigquery-public-data",
                                  dataset_name="github_repos")

We'll work with two tables in the database.  The first table is the "licenses" table, which provides the name of each GitHub repo (in the `repo_name` column) and its corresponding license.

In [ ]:
github.head("licenses")

The second table is the "sample_files" table, which provides, among other information, the GitHub repo that each file belongs to (in the `repo_name` column).

In [ ]:
github.head("sample_files")

Next, we write a query that leverages the information in both tables to determine how many files are released in each license.

In [ ]:
query = ("""
        SELECT L.license, COUNT(1) AS number_of_files
        FROM `bigquery-public-data.github_repos.sample_files` as sf
        INNER JOIN `bigquery-public-data.github_repos.licenses` as L 
            ON sf.repo_name = L.repo_name
        GROUP BY L.license
        ORDER BY number_of_files DESC
        """)

file_count_by_license = github.query_to_pandas_safe(query, max_gb_scanned=6)

It's a big query, and so we'll investigate each piece separately.

![](https://i.imgur.com/b2H4RDn.png)
    
We'll begin with the **JOIN** (highlighted in blue above).  This specifies the sources of data and how to join them. We use **ON** to specify that we use the `repo_name` columns for the **JOIN**. This join will allow us to select columns from both tables!

Next, we'll talk about the **SELECT** and **GROUP BY** parts (highlighted in yellow).  The **GROUP BY** breaks the data into a different group for each license, before we **COUNT** the number of rows in the "sample_files" table that corresponds to each license.  (Remember that you can count the number of rows with `COUNT(1)`.) 

Finally, the **ORDER BY** (highlighted in purple) specifies the order when showing the results.

It was a big query, but it gave us a nice table summarizing how many files have been committed under each license:  

In [ ]:
print(file_count_by_license)

You'll use **JOIN** clauses a lot and get very efficient with them as you get some practice.

# Your Turn

You are on the last step.  Finish it by solving **[this question](#$NEXT_NOTEBOOK_URL$)**.